<a href="https://colab.research.google.com/github/prithuchauhan/Audio-DL/blob/main/Music_classifier_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader,Dataset
import torchvision
import torchaudio
import torchaudio.transforms as transfroms
import soundfile as sf
import transformers



#gtzan is a popular dataset for music genre classification.

In [ ]:
!pip install -U datasets

In [ ]:
from datasets import load_dataset

gtzan = load_dataset("marsyas/gtzan", "all")
gtzan

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 999
    })
})

In [ ]:
def reduce_dataset_size(dataset, fraction=0.1):
    num_samples = int(len(dataset) * fraction)
    return dataset.shuffle(seed=42).select(range(num_samples))

In [ ]:
# create a test train split

gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)
gtzan

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

In [ ]:
gtzan['train'] = reduce_dataset_size(gtzan['train'], fraction=0.1)
gtzan

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 89
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

In [ ]:
gtzan['test'] = reduce_dataset_size(gtzan['test'], fraction=0.1)
gtzan

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 89
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 10
    })
})

In [ ]:
gtzan["train"][0]

{'file': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/hiphop/hiphop.00037.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/hiphop/hiphop.00037.wav',
  'array': array([0.        , 0.        , 0.        , ..., 0.00576782, 0.02093506,
         0.01660156]),
  'sampling_rate': 22050},
 'genre': 4}

In [ ]:
gtzan["train"][1]

{'file': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/classical/classical.00070.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/classical/classical.00070.wav',
  'array': array([-0.01803589, -0.02142334, -0.01879883, ...,  0.05975342,
          0.06149292,  0.06118774]),
  'sampling_rate': 22050},
 'genre': 1}

In [ ]:
idlabel = gtzan["train"].features["genre"].int2str
idlabel(gtzan["train"][0]["genre"])

'hiphop'

In [ ]:
idlabel(gtzan["train"][1]["genre"])

'classical'

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr


def generate_audio():
    example = gtzan["train"].shuffle()[0]
    audio = example["audio"]
    return (
        audio["sampling_rate"],
        audio["array"],
    ), idlabel(example["genre"])


with gr.Blocks() as demo:
    with gr.Column():
        for _ in range(2):
            audio, label = generate_audio()
            output = gr.Audio(audio, label=label)

demo.launch(debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/processing_utils.py:574: UserWarning: Trying to convert audio automatically from float64 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://3dbae6336dcaa27147.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://3dbae6336dcaa27147.gradio.live


DistilHuBERT is a distilled version of HuBERT we will be using this for audio classification. It is 73% faster than HuBERT with similar performance


In [ ]:
from transformers import AutoFeatureExtractor

model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

In [ ]:
smpling_rate = feature_extractor.sampling_rate
smpling_rate

16000

In [ ]:
from datasets import Audio
gtzan = gtzan.cast_column("audio",Audio(sampling_rate = smpling_rate))

In [ ]:
gtzan["train"][0]

{'file': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/hiphop/hiphop.00037.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/hiphop/hiphop.00037.wav',
  'array': array([0.        , 0.        , 0.        , ..., 0.01601283, 0.01678947,
         0.        ]),
  'sampling_rate': 16000},
 'genre': 4}

In [ ]:
sample = gtzan["train"][0]["audio"]

print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")          # calculates mean and variance of the array of the first sample in gtzan dataset

Mean: -9.52e-06, Variance: 0.0413


In [ ]:
 inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])

print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)

inputs keys: ['input_values', 'attention_mask']
Mean: 2.53e-09, Variance: 1.0


here input to feature extractor is our array of amplitude values and the sampling rate , the output as we see is a dict two arrays input values and attention mask. input values are preprocessed audio inputs u can see their mean is actually very close to zero. The attention_mask is used when we process a batch of audio inputs at once - it is used to tell the model where we have padded inputs of different lengths. after the preprocessing we can see the mean is 0 and variance is 1 - exactly what we expected. (or the model expected to be fed).

In [ ]:
max_duration = 30.0


def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

truncating our samples to 30s each.

In [ ]:



gtzan_encoded = gtzan.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=100,
    num_proc=1,
)
gtzan_encoded



# map can be batched as given by saving batched = true. default batch size is 1000.

Map:   0%|          | 0/89 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 89
    })
    test: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 10
    })
})

In [ ]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")

In [ ]:
id2label = {
    str(i): idlabel(i)
    for i in range(len(gtzan_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

id2label["7"]

'pop'

In [ ]:
from transformers import AutoModelForAudioClassification

num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install accelerate -U

In [ ]:
!pip install transformers[torch] accelerate -U


In [ ]:
# fine tuning a model using trainer.

from transformers import TrainingArguments

model_name = model_id.split("/")[-1]
batch_size = 2
gradient_accumulation_steps = 4
num_train_epochs = 2

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=False,
)

In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00


In [ ]:
# using accuracy as a metric
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,2.311600,2.255078,0.300000
1,2.252100,2.215234,0.300000


TrainOutput(global_step=22, training_loss=2.2811945134943183, metrics={'train_runtime': 141.0595, 'train_samples_per_second': 1.262, 'train_steps_per_second': 0.156, 'total_flos': 1.1940466608e+16, 'train_loss': 2.2811945134943183, 'epoch': 1.9555555555555557})

training would actually takes hours hence only two epochs.

In [ ]:
test_results = trainer.evaluate(eval_dataset=gtzan_encoded["test"])

print("Test Results:", test_results)

Test Results: {'eval_loss': 2.255078077316284, 'eval_accuracy': 0.3, 'eval_runtime': 11.1004, 'eval_samples_per_second': 0.901, 'eval_steps_per_second': 0.45, 'epoch': 1.9555555555555557}


Accuracy is 0.3 means it got 3 right out of 10 in the test set. Accuracy is ofcourse expected to be low due to only 2 epochs of training.